In [ ]:
# 获取数据
import requests
import json
china_url = "https://view.inews.qq.com/g2/getOnsInfo?name=disease_h5"
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36'
}
# 获取到json数据
response = requests.get(url=china_url,headers=headers).json()
# 先把json数据转为python数据格式
data = json.loads(response['data'])
# 保存数据
with open('./国内疫情.json','w') as f:
    # 再把python格式数据转成json
    #indent参数表示空格的缩进
    #ensure_ascii,编码改为False,不想ascii编码，想要正常编码
    f.write(json.dumps(data,ensure_ascii=False,indent=2))

In [18]:
import json
import pandas as pd
# 将json数据转存到excel
with open('./国内疫情.json','r') as f:
    data = f.read()
# 将字符串格式转换为python数据格式
data = json.loads(data)
# 获取国内所有数据
chinaAreaDict = data['areaTree'][0]
# 获取所有省份数据
provinceList = chinaAreaDict['children']
# print(len(provinceList))
china_cityList = [] # [{湖北，武汉},{湖北，襄阳}]
for x in range(len(provinceList)):
    # 获取每一个省份
    province = provinceList[x]['name']
    # 一个省中有多少个市
    province_list = provinceList[x]['children']
    for y in range(len(province_list)):
        # 每一个城市的数据
        city = province_list[y]['name']
        today = province_list[y]['today']
        total = province_list[y]['total']
        city_dict = {'province':province,
                    'city':city,
                    'today':today,
                    'total':total}
        china_cityList.append(city_dict)
# 中国所有城市的总体数据
chinaTotalData = pd.DataFrame(china_cityList)
# 将chinaTotalData中的today和total数据添加到dataframe中

# 拆分total列
confirmlist = []
suspectlist = []
deadlist = []
heallist = []
deadRatelist = []
healRatelist = []
for value in chinaTotalData['total'].values.tolist():
    confirmlist.append(value['confirm'])
    suspectlist.append(value['suspect'])
    deadlist.append(value['dead'])
    heallist.append(value['heal'])
    deadRatelist.append(value['deadRate'])
    healRatelist.append(value['healRate'])

chinaTotalData['confirm'] = confirmlist
chinaTotalData['suspect'] = suspectlist
chinaTotalData['dead'] = deadlist
chinaTotalData['heal'] = heallist
chinaTotalData['deadRate'] = deadRatelist
chinaTotalData['healRate'] = healRatelist

# 拆分today列
today_confirmlist = []
today_confirmCutslist = []
for value in chinaTotalData['today'].values.tolist():
    today_confirmlist.append(value['confirm'])
    today_confirmCutslist.append(value['confirmCuts'])
    
chinaTotalData['today_confirm'] = today_confirmlist
chinaTotalData['today_confirmCuts'] = today_confirmCutslist

# 删除total列,axis=1参数表示以列删除，inplace=True，表示在原有的基础上删除
chinaTotalData.drop(['total','today'],axis=1,inplace=True)

# 将其保存到excel中
from openpyxl import load_workbook
# 这样写可以避免对原来的数据进行覆盖
book = load_workbook('国内疫情.xlsx')
# pd.ExcelWriter保存结果到已存在的excel文件中
writer = pd.ExcelWriter('国内疫情.xlsx',engine='openpyxl')
writer.book = book
writer.sheets = dict((ws.title,ws)for ws in book.worksheets)
chinaTotalData.to_excel(writer,index=False)
writer.save()
writer.close()
        

In [22]:
# 绘制柱状图
from pyecharts.charts import Bar
bar = Bar()
bar.add_xaxis(['衬衫','羊毛衫','雪纺衫','裤子','高跟鞋','袜子'])
bar.add_yaxis("商家A",[5,20,36,10,75,90])
# render会生成本地HTML文件，默认会在当前目录生成render.html
# 也可以传入路径参数，如bar.render("my.html")
# 也可以将图形在jupyter输出，如bar.render_notebook()
bar.render()

'D:\\File\\My_Jupyter\\analysis_test\\render.html'

In [25]:
import pandas as pd
from pyecharts import options as opts
from pyecharts.charts import Map
df = pd.read_excel('./国内疫情.xlsx')
# 1.根据绘制国内疫情图（确诊）
# 按照身份分组然后作和
data = df.groupby(by='province',as_index=False).sum() #dataframe数据
# 形成这样的数据[(黑龙江，600),(吉林，50)]
data_list = list(zip(data['province'].values.tolist(),data['confirm'].values.tolist()))

def map_china() -> Map:
    c = (
        Map()
        .add(series_name='确诊病例',data_pair=data_list,maptype="china")
        .set_global_opts(
            title_opts=opts.TitleOpts(title="疫情地图"),
            visualmap_opts=opts.VisualMapOpts(is_piecewise=True,
                pieces=[{"max":9,"min":0,"lable":"0-9","color":"#FFE4E1"},
                       {"max":99,"min":10,"lable":"10-99","color":"#FF7F50"},
                       {"max":499,"min":100,"lable":"100-499","color":"#F08080"},
                       {"max":999,"min":500,"lable":"500-999","color":"#CD5C5C"},
                       {"max":9999,"min":1000,"lable":"1000-9999","color":"#990000"},
                       {"max":99999,"min":10000,"lable":"10000-99999","color":"#660000"},]
                                             )
        )
    )
    return c
d_map = map_china()
d_map.render_notebook()